### Approach- <br><br>1. Data reading and understanding<br>2. EDA[EDA shown here is only for one specific store and dept, this behavior was observed in most data]<br>3. Data preparation and feature generation<br>4. Data modelling<br> => As evident from EDA plots, seasonality is present, so regression is built with past year sale coming out to be an important feature<br> => FBProphet to capture seasonality automatically<br> => Seasonal differencing and ARIMA(AR1(1)) to capture correlation at lag 1<br>5. Average the predictions from each of the model

In [ ]:
#All imports
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
from datetime import datetime
from zipfile import ZipFile

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#preprocessing
from sklearn.preprocessing import StandardScaler

#modelling - regression
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

#modelling - timeseries
from fbprophet import Prophet
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller

#evaluation
from sklearn.metrics import mean_absolute_error

In [ ]:
SOURCE_PATH='../input/walmart-recruiting-store-sales-forecasting/'

## **Step 1 - Data Reading and Understanding**

In [ ]:
#extract all the zip files
for root, dirs, files in os.walk(SOURCE_PATH):
    for file_name in files:
        if(file_name.endswith('.zip')):
            file_path = SOURCE_PATH + file_name
            with ZipFile(file_path, 'r') as zip_file_handle:
                zip_file_handle.extractall()

In [ ]:
#Read train data
train_df = pd.read_csv('train.csv')
train_df['Date'] = pd.to_datetime(train_df['Date'], format='%Y-%m-%d')
train_df.head()

In [ ]:
train_df.info()

In [ ]:
min_date = train_df['Date'].min()
max_date = train_df['Date'].max()
n_train_samples = train_df.shape[0]

print ("Train Data ranges from {} to {}".format(min_date, max_date))
print ("# of train samples: ", n_train_samples)

In [ ]:
#Read test data
test_df = pd.read_csv('./test.csv')
test_df['Date'] = pd.to_datetime(test_df['Date'], format='%Y-%m-%d')
test_df.head()

In [ ]:
min_date = test_df['Date'].min()
max_date = test_df['Date'].max()
n_test_samples = test_df.shape[0]

print ("Test Data ranges from {} to {}".format(min_date, max_date))
print ("# of train samples: ", n_test_samples)

**Concatenate train and test data to ease on feature generation**

In [ ]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
print(all_df.shape)
print(all_df.head())

In [ ]:
#Read features data
features_df = pd.read_csv('features.csv')
features_df['Date'] = pd.to_datetime(features_df['Date'])
features_df.head()

**Combining Sales,features, stores data on the basis of Store and Date**

In [ ]:
all_df = pd.merge(all_df, features_df, how='inner', on=['Store', 'Date', 'IsHoliday'])
all_df.head()

In [ ]:
#Read stores data
stores_df = pd.read_csv(SOURCE_PATH + 'stores.csv')
stores_df.head()

**Combining Stores details**

In [ ]:
all_df = pd.merge(all_df, stores_df, how='inner', on='Store')
all_df.head()

In [ ]:
#Check if any duplicate records with respect to Store and Dept
all_df[all_df.duplicated(subset=['Store', 'Dept', 'Date'])]

In [ ]:
#checking the distribution of numerical columns like Temperature, FuelPrice..
all_df[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']].describe()

## **Step 2 - EDA**

**Extracting few fields to ease on EDA**

In [ ]:
#Extracting Year, Month, Date
all_df['Year'] = all_df['Date'].dt.year
all_df['Month'] = all_df['Date'].dt.month
all_df['WeekNumber'] = all_df['Date'].dt.week

#Extracting WeekendNumber for each month
all_df['WeekendNumber'] = all_df.groupby(by=['Store','Year','Month'])['Date'].rank(method='dense').astype('int')
all_df.head()

**Distribution of Weekly_Sales**

In [ ]:
sns.distplot(all_df['Weekly_Sales'])
plt.show()

**From the distribution plot, data is highly skewed. Also, Scaling would be needed for regression to bring features on same values range**

**For further EDA, taking just one store, one department data. The same can be repeated for others too**

In [ ]:
store_1_dep_1_df = all_df[(all_df.Store ==1) & (all_df.Dept==1)]
print(store_1_dep_1_df.shape)
store_1_dep_1_df.head()

In [ ]:
#Overall sales plot to see if any trend present
plt.plot(store_1_dep_1_df['Date'], store_1_dep_1_df['Weekly_Sales'])
plt.title("3 yr Sales of Store 1 and Dept 1")
plt.xticks(rotation=45)
plt.show()

#Sales for each year to capture any seasonality
yr=2010
while(yr<=2012):
    sales_per_year = store_1_dep_1_df[store_1_dep_1_df.Year==yr]
    plt.plot(sales_per_year['Date'], sales_per_year['Weekly_Sales'])
    plt.title("Sales of Store 1 and Dept 1 - Year {}".format(yr))
    plt.xticks(rotation=45)
    plt.show()
    
    yr+=1

**There is no apparent trend but certainly there is seasonality. And the peaks corresponding to Holiday season - Feb(Superbowl), Nov(Thanksgiving), Dec(Christmas Month)**

**December month has a special behavior - Sales are high on all weekends before Christmas eve. This can be seen from plot and values**

In [ ]:
print("Year - 2010", store_1_dep_1_df[(store_1_dep_1_df.Year==2010) & (store_1_dep_1_df.Month==12)]['Weekly_Sales'])
print("Year - 2011", store_1_dep_1_df[(store_1_dep_1_df.Year==2011) & (store_1_dep_1_df.Month==12)]['Weekly_Sales'])

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(store_1_dep_1_df[(store_1_dep_1_df.Year==2010) & (store_1_dep_1_df.Month==12)]['WeekendNumber'], store_1_dep_1_df[(store_1_dep_1_df.Year==2010) & (store_1_dep_1_df.Month==12)]['Weekly_Sales'], label='2010')
plt.plot(store_1_dep_1_df[(store_1_dep_1_df.Year==2011) & (store_1_dep_1_df.Month==12)]['WeekendNumber'], store_1_dep_1_df[(store_1_dep_1_df.Year==2011) & (store_1_dep_1_df.Month==12)]['Weekly_Sales'], label='2011')
plt.xticks([1,2,3,4,5])
plt.legend(loc='upper right')    
plt.show()

## **Step 3 - Data Preparation**

**Replacing column: Type and Size with an ordered numeric indicator about the size of column<br>A/1 => Big stores, B/2 => Medium size stores, C/3 => Small stores**

In [ ]:
all_df['StoreSize'] = all_df['Type'].map({'A':1, 'B':2, 'C':3})
all_df.drop(columns=['Type', 'Size'], inplace=True)

all_df.head()

**Drop columns => Temperature, Fuel_Price, MarkDown*, CPI, Unemployment**

In [ ]:
cols_to_drop = [c for c in all_df.columns if c[0:8] == 'MarkDown']
cols_to_drop.extend(['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'])
all_df.drop(columns=cols_to_drop, inplace=True)
all_df.head()

**Create new columns from IsHoliday as indicator for days when sales are high.<br>IsSuperbowl => superbowl weekend<br>IsThanksgiving => Thanksgiving weekend<br>IsChristmasMonth => weekends prior to christmas weekend**

In [ ]:
#Sales are high on superbowl weekend
all_df['IsSuperbowl'] = 0
all_df.loc[all_df['Date'].isin(['2010-02-12','2011-02-11','2012-02-10','2013-02-08']),'IsSuperbowl'] = 1

#Sales are high on Thanksgiving weekend
all_df['IsThanksgiving'] = 0
all_df.loc[all_df['Date'].isin(['2010-11-26','2011-11-25','2012-11-23','2013-11-29']), 'IsThanksgiving'] = 1

#Sales are high in month of december on all weekends prior to christmas weekend
all_df['IsChristmasMonth'] = 0
all_df.loc[(all_df['Month']==12) & (all_df['Year']==2010) & (all_df['Date'] < '2010-12-31'), 'IsChristmasMonth'] = 1
all_df.loc[(all_df['Month']==12) & (all_df['Year']==2011) & (all_df['Date'] < '2011-12-30'), 'IsChristmasMonth'] = 1
all_df.loc[(all_df['Month']==12) & (all_df['Year']==2012) & (all_df['Date'] < '2012-12-28'), 'IsChristmasMonth'] = 1
all_df.loc[(all_df['Month']==12) & (all_df['Year']==2013) & (all_df['Date'] < '2013-12-27'), 'IsChristmasMonth'] = 1

**From EDA, Sales are high on weekend in the beginning of the month. In case, first weekend falls on 1st or 2nd of the month, then sales would be high on next weekend.**

In [ ]:
#Pull out Date and WeekendNumber to decide on high sales weekend and this can be later merged with all_df
def isHighSaleWeekend(row):
  if row['WeekendNumber'] == 1 and row['Date'].date().day < 3:
    return 0
  elif row['WeekendNumber'] == 1 and row['Date'].date().day >= 3:
    return 1
  elif row['WeekendNumber'] == 2 and ( (row['Date'].date().day==8)  or (row['Date'].date().day==9) ):
    return 1
  else:
    return 0

all_date_df = all_df[['Date', 'WeekendNumber']]
all_date_df.drop_duplicates(inplace=True, ignore_index=True)
all_date_df['IsHighSaleWeekend'] = all_date_df.apply(isHighSaleWeekend, axis=1)
all_date_df.head()

In [ ]:
#Adding IsHighSaleWeekend feature to indicate high sales in beginning of the month
all_df = pd.merge(all_df, all_date_df, on=['Date','WeekendNumber'], how='inner')
all_df.head() 

**As evident from plot above that data has seasonality, so adding past year sale as feature corresponding to Store, Dept**

In [ ]:
all_grouped_df = all_df.groupby(['Store','Dept','Year'])
all_keys = all_grouped_df.groups.keys()
done = set()
for k in all_keys:
    if (k[0], k[1]) not in done:
        s = k[0]
        d = k[1]
        yr = 2011        
        while(yr <= 2013):            
            try:
                sales_yr = all_grouped_df.get_group((k[0],k[1],yr))
                sales_past_yr = all_grouped_df.get_group((k[0],k[1],yr-1))
                
                for _,row in sales_yr.iterrows():
                    week_number = row['WeekNumber']
                    last_yr_sale = sales_past_yr[sales_past_yr.WeekNumber==week_number]['Weekly_Sales']                    
                    if(len(last_yr_sale) > 0):
                        all_df.loc[(all_df.Store==s) & (all_df.Dept==d) & (all_df.Year==yr) & (all_df.WeekNumber==week_number), 'Weekly_Sales_past1yr'] =  last_yr_sale.values[0]                        
            except KeyError as exc:
                print(exc)
                
            yr+=1
                
        done.add((k[0], k[1]))

**For Year 2010 and till WeekNumber of 5 for Year 2011, no past year data is available so not considering them for imputation of past year sales** 

In [ ]:
all_with_missing_df = all_df[~(all_df.Year==2010)]
all_with_missing_df = all_with_missing_df[~((all_df.Year==2011) & (all_df.WeekNumber < 5))]
all_with_missing_df = all_with_missing_df[all_with_missing_df['Weekly_Sales_past1yr'].isnull()]
all_with_missing_df.shape

In [ ]:
#iterate over each row and use average to impute the missing past year sale
#first compute average sale of past year, same month for corresponding store and dept
#if not present, then compute average of all data for past year for corresponding store and dept
#else substitute with 0

for _, row in all_with_missing_df.iterrows():
    s = row['Store']
    d = row['Dept']
    y = row['Year']
    m = row['Month']
    w = row['WeekNumber']
    
    past_sale = all_df[(all_df.Store==s) & (all_df.Dept==d) & (all_df.Year==y-1) & (all_df.Month==m)]['Weekly_Sales']           
    if(len(past_sale) > 0 and not np.isnan(past_sale.mean())):
        all_df.loc[(all_df.Store==s) & (all_df.Dept==d) & (all_df.Year==y) & (all_df.Month==m) & (all_df.WeekNumber==w), 'Weekly_Sales_past1yr'] = past_sale.mean()                
    else:
        past_sale = all_df[(all_df.Store==s) & (all_df.Dept==d) & (all_df.Year==y-1)]['Weekly_Sales']        
        if(len(past_sale) > 0 and not np.isnan(past_sale.mean())):
            all_df.loc[(all_df.Store==s) & (all_df.Dept==d) & (all_df.Year==y) & (all_df.Month==m) & (all_df.WeekNumber==w), 'Weekly_Sales_past1yr'] = past_sale.mean()            
        else:            
            all_df.loc[(all_df.Store==s) & (all_df.Dept==d) & (all_df.Year==y) & (all_df.Month==m) & (all_df.WeekNumber==w), 'Weekly_Sales_past1yr'] = 0                                        

In [ ]:
all_df.info()

## **Step5 - Data Modelling**

In [ ]:
#helper function to compute weighted mean absolute error
def compute_weighted_mae(true, pred, weight):
    return mean_absolute_error(y_true=true, y_pred=pred, sample_weight = weight)

In [ ]:
#To bring features on same scale, so using StandardScaler
scaler = StandardScaler()
all_df[['Weekly_Sales_Scaled']] = scaler.fit_transform(all_df[['Weekly_Sales']])
all_df[['Weekly_Sales_Past1yr_Scaled']] = scaler.fit_transform(all_df[['Weekly_Sales_past1yr']])

In [ ]:
#split train and test data
train_data = all_df.loc[0:n_train_samples-1]
train_data = train_data[~(train_data.Year==2010)]   #Discard 2010 data as there is no information on past yr sales
train_data = train_data[~( (train_data.Year==2011) & (train_data.WeekNumber < 5) )] #No data from year 2010 prior to WeekNumber 5

test_data = all_df.loc[n_train_samples:]

### **Model 1 - Linear Regression**

In [ ]:
#prepare train and validation data
X = train_data[['StoreSize', 'IsSuperbowl', 'IsThanksgiving', 'IsChristmasMonth','IsHighSaleWeekend','Weekly_Sales_Past1yr_Scaled']]
y = train_data['Weekly_Sales_Scaled']

n_split = int(len(X)*0.8)
X_train, X_val = X[0:n_split], X[n_split:]
y_train, y_val = y[0:n_split], y[n_split:]
X_train_weight = train_data['IsHoliday'][0:n_split].map({True:5, False:1})
X_val_weight = train_data['IsHoliday'][n_split:].map({True:5, False:1})

print("# of training samples: ", len(X_train))
print("# of validation samples: ", len(X_val))

In [ ]:
#fit linear regression on train split data
X_train_sm = sm.add_constant(X_train)
lr_model = sm.OLS(y_train, X_train).fit()
print(lr_model.summary())

**From R-square, 96.6% of variance can be explained by this model**

In [ ]:
#run predictions on train and val data and compute weighted mean
y_train_pred = lr_model.predict(X_train)
y_train_inv = scaler.inverse_transform(y_train.values.reshape(-1,1))
y_train_pred_inv = scaler.inverse_transform(y_train_pred.values.reshape(-1,1))
train_wmae = compute_weighted_mae(y_train_inv[:,0], y_train_pred_inv[:,0], X_train_weight)
print("Train wmae: ", train_wmae)

y_val_inv = scaler.inverse_transform(y_val.values.reshape(-1,1))
y_val_pred = lr_model.predict(X_val)
y_val_pred_inv = scaler.inverse_transform(y_val_pred.values.reshape(-1,1))
val_wmae = compute_weighted_mae(y_val_inv, y_val_pred_inv, X_val_weight)
print("Validation wmae: ", val_wmae)

**Test Prediction**

In [ ]:
X_test = test_data[['StoreSize', 'IsSuperbowl', 'IsThanksgiving', 'IsChristmasMonth','IsHighSaleWeekend','Weekly_Sales_Past1yr_Scaled']]

print("# of samples in test data: ", X_test.shape[0])

y_test_pred = lr_model.predict(X_test)
lr_predictions = scaler.inverse_transform(y_test_pred.values.reshape(-1,1))
test_data['Predicted_Sales_reg'] = lr_predictions

### **Model 2 - TimeSeries using FbProphet**

In [ ]:
test_data['Predicted_Sales_fbprophet'] = np.nan

print("# of training samples: ", len(train_data))
print("# of testing samples: ", len(test_data))

In [ ]:
def predict_sales(store_dept_data, store_dept_test):
    s = store_dept_data['Store'].values[0]
    d = store_dept_data['Dept'].values[0]
    
    if(len(store_dept_data) < 2):
        print("There are less than 2 records for store:{} and dept:{}, skipping prediction".format(s,d))
        return
    
    store_dept_train = store_dept_data[['Date','Weekly_Sales','IsSuperbowl','IsThanksgiving',
                                        'IsChristmasMonth','IsHighSaleWeekend','Weekly_Sales_past1yr']]
    store_dept_train.rename(columns={'Date':'ds', 'Weekly_Sales':'y'}, inplace=True)
        
    store_dept_test.rename(columns={'Date':'ds'}, inplace=True)
    store_dept_test.reset_index(inplace=True, drop=True)
    
    #holidays = store_dept_data[store_dept_data['IsHoliday']==True]['Date']
    #holidays_df = pd.DataFrame({'holiday':'festive season', 'ds':holidays})
    
    prophet = Prophet(yearly_seasonality=True, weekly_seasonality=True, interval_width=0.95)
    prophet.add_regressor('IsSuperbowl')
    prophet.add_regressor('IsThanksgiving')
    prophet.add_regressor('IsChristmasMonth')
    prophet.add_regressor('IsHighSaleWeekend')
    prophet.add_regressor('Weekly_Sales_past1yr')
    
    try:
        prophet.fit(store_dept_train)
        forecast = prophet.predict(store_dept_test)
        predictions = forecast['yhat']
        for i in range(len(store_dept_test)):          
            yr = store_dept_test.loc[i,'Year']
            mth = store_dept_test.loc[i,'Month']
            wn = store_dept_test.loc[i,'WeekNumber']

            test_data.loc[((test_data.Store==s) & (test_data.Dept==d) & (test_data.Year==yr) & (test_data.Month==mth) & (test_data.WeekNumber==wn)),'Predicted_Sales_fbprophet'] = predictions[i]             
    except:
        print("Exception processing store:{} and dept: {}".format(s,d))        

In [ ]:
test_grouped_data = test_data.groupby(['Store','Dept'])
train_grouped_data = train_data.groupby(['Store','Dept'])

for key in train_grouped_data.groups.keys():
    if key in test_grouped_data.groups.keys():
        predict_sales(train_grouped_data.get_group(key), test_grouped_data.get_group(key))

### **Model 3 - ARIMA**

In [ ]:
test_data['Predicted_Sales_arima'] = np.nan

print("# of training samples: ", len(train_data))
print("# of testing samples: ", len(test_data))

**Using data of Store 1 and Dept 1 to decide on ARIMA order**

In [ ]:
#Perform seasonal differencing and check ACF, PACF
store_1_dept_1_df['Differenced_Sales'] = store_1_dept_1_df['Weekly_Sales'] - store_1_dept_1_df['Weekly_Sales_past1yr']

#test to check if time series is stationary
result = adfuller(store_1_dept_1_df['Differenced_Sales'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print("{}:{}".format(key, value))
    
#plots to check if any level of correlation is present with lags
plot_acf(store_1_dept_1_df['Differenced_Sales'])
plt.show()

plot_pacf(store_1_dept_1_df['Differenced_Sales'])
plt.show()

**As positive correlation is present, using AR1 to consider the same**

In [ ]:
def predict_sales_arima(store_dept_data, train_order, store_dept_test):
    s = store_dept_data['Store'].values[0]
    d = store_dept_data['Dept'].values[0]
        
    store_dept_test.reset_index(inplace=True, drop=True)
        
    #seasonal differencing to make time series stationary
    diff = store_dept_data['Weekly_Sales'] - store_dept_data['Weekly_Sales_past1yr']        
    
    try:
        model = ARIMA(diff, order=train_order)
        model_fit = model.fit(trend='nc')
    except:
        print("Exception fitting ARIMA on store:{} and dept:{}, so skipping".format(s,d))
        return
    #print(model_fit.summary())
    
    pred_start_idx = len(store_dept_data)
    pred_end_idx = len(store_dept_data) + len(store_dept_test)-1
    
    test_predictions = model_fit.predict(start=pred_start_idx, end=pred_end_idx, dynamic=True)    
    test_predictions.reset_index(inplace=True, drop=True)
            
    for i in range(len(store_dept_test)):        
        test_predictions[i] += store_dept_test.loc[i,'Weekly_Sales_past1yr']
        
        yr = store_dept_test.loc[i,'Year']
        mth = store_dept_test.loc[i,'Month']
        wn = store_dept_test.loc[i,'WeekNumber']
        test_data.loc[((test_data.Store==s) & (test_data.Dept==d) & (test_data.Year==yr) & (test_data.Month==mth) & (test_data.WeekNumber==wn)),'Predicted_Sales_arima'] = test_predictions[i]             

In [ ]:
test_grouped_data = test_data.groupby(['Store','Dept'])
train_grouped_data = train_data.groupby(['Store','Dept'])
store_dept_data = train_grouped_data.get_group((1,1))

for key in train_grouped_data.groups.keys():
    if key in test_grouped_data.groups.keys():
        predict_sales_arima(train_grouped_data.get_group(key), (1,0,0), test_grouped_data.get_group(key))

## **Step 6 - Submission**

In [ ]:
def compute_average_sales(row):
    values = [row['Predicted_Sales_reg']]
    
    if not np.isnan(row['Predicted_Sales_fbprophet']):
        values.append(row['Predicted_Sales_fbprophet'])        
        
    if not np.isnan(row['Predicted_Sales_arima']):
        values.append(row['Predicted_Sales_arima'])
        
    return np.mean(values) 

In [ ]:
#combining the results of regression ,arima and fbprophet
test_data['Final_Predicted_Sales'] = test_data.apply(lambda x:compute_average_sales(x), axis=1)    
print(test_data.shape)
print(test_data.head())
    
id = test_data.apply(lambda row:str(row['Store']) + "_" + str(row['Dept']) + "_" + (row['Date']), axis=1)
submission_df = pd.DataFrame({'Id':id, 'Weekly_Sales':test_data['Final_Predicted_Sales']})    
print(submission_df.shape)
print(submission_df.head())

In [ ]:
submission_df.to_csv('./Submission.csv', index=False)